In [1]:
import tensorflow  as tf

In [2]:
tf.__version__

'2.2.0'

In [3]:
# 因为我们在梯度下降的时候需要使用到变量以及存储
# 创建变量
v = tf.Variable(0.0)

In [4]:
(v + 1).numpy()

1.0

In [5]:
# 改变这个变量方法一
v.assign(5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [6]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [7]:
# 改变这个变量方法二
v.assign_add(1)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=6.0>

In [8]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>

In [9]:
v.read_value()

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [10]:
# 网络训练过程中就求解梯度，导数的方向就是变化最快的方向
# 也就是求解微分
# GradientTape会自动跟踪这些变量的变化
w = tf.Variable([[1.0]])
with tf.GradientTape() as t:
    loss = w * w

In [11]:
# t记录后调用gradient求解loss对于w的微分
grad = t.gradient(loss, w)

In [12]:
grad # 结果为2 

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [13]:
# 注意是浮点型
w = tf.constant(3.0)
with tf.GradientTape() as t:
    # 让这个t 去跟踪常量的运算
    t.watch(w)
    loss = w * w

In [14]:
# 求出其具体的微分
dloss_dw = t.gradient(loss, w)

In [15]:
dloss_dw # x的平方 = 2x

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [16]:
# 如果需要计算多次微分的话，需要设置为True,不然的话后面z的求解会丢失
w = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(w)
    y = w * w
    z = y * y

In [17]:
dy_dw = t.gradient(y, w)

In [18]:
dy_dw

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [19]:
dz_dw = t.gradient(z, w)

In [20]:
dz_dw

<tf.Tensor: shape=(), dtype=float32, numpy=108.0>

# 简单的例子

In [21]:
# 如果测试用不到的话
# (train_image, train_labels), #
(train_image, train_labels), (test_image, test_labels) = tf.keras.datasets.mnist.load_data()

In [22]:
train_image.shape

(60000, 28, 28)

In [23]:
# 扩征维度，最后一个维度
train_image = tf.expand_dims(train_image, -1)

In [24]:
test_image = tf.expand_dims(test_image, -1)

In [25]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [26]:
# 改变数据类型，以及预处理
train_image = tf.cast(train_image/255, tf.float32)

In [27]:
test_image = tf.cast(test_image/255, tf.float32)

In [28]:
train_labels = tf.cast(train_labels, tf.int64)

In [29]:
test_labels = tf.cast(test_labels, tf.int64)

In [30]:
# tf.data模块
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_labels))

In [31]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_labels))

In [32]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [33]:
# 乱序，批次大小32
dataset = dataset.shuffle(10000).batch(32)

In [34]:
test_dataset = test_dataset.batch(32)

In [35]:
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

## 自定义训练

In [36]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], activation='relu', input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(32, [3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10)
])

In [37]:
# 自定义这个循环就不编译了，我们设置优化器
optimizer = tf.keras.optimizers.Adam()

In [38]:
# 计算loss函数   顺序编码
# 没有激活 fromlogits为True
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [39]:
# 因为tf.data模块可以直接迭代的，取出生成器的下个
features, labels = next(iter(dataset))

In [40]:
# batch 32
features.shape

TensorShape([32, 28, 28, 1])

In [41]:
labels.shape

TensorShape([32])

In [42]:
# 内置了keras方法所以可调用
predictions = model(features)

In [43]:
predictions.shape

TensorShape([32, 10])

In [44]:
# 通过结果可知，确实做出了预测，但是不准，需要训练
tf.argmax(predictions, axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7], dtype=int64)>

In [45]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([4, 7, 8, 2, 0, 4, 4, 3, 5, 7, 7, 7, 8, 1, 5, 7, 5, 1, 1, 4, 9, 8,
       4, 7, 5, 3, 7, 6, 6, 3, 9, 4], dtype=int64)>

In [46]:
# 定义一个损失函数 每一步的损失值
"""
    loss_func(y_true, y_pred, sample_weight=None)
"""
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [47]:
# 初始化两个计算对象 tf.keras.metrics 汇总计算模块
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [48]:
# 一步的train 对于一个batch
def train_step(model, images, labels):
    # 在梯度下计算我们的损失函数
    with tf.GradientTape() as t:
        # 上面的函数loss，我们拆开了
        pred = model(images)
        loss_step = loss_func(labels, pred)
    # model的训练参数 与损失函数之间的梯度
    grads = t.gradient(loss_step, model.trainable_variables)
    # 使用这个方法，来改变这个变量数值，使他下降的最快
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # 每一步进行进入 后面会进行统计tf.keras.metrics 汇总计算模块
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [49]:
def test_step(model, images, labels):
    pred = model(images)
    loss_step = loss_func(labels, pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)

In [50]:
def train():
    for epoch in range(10):
        # batch就是enumerate给他加的序号0123
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels) #一步训练
        print('Epoch{} loss is {}, accuracy is {}'.format(epoch,
                                                          train_loss.result(),
                                                          train_accuracy.result()))
        
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(model, images, labels)
        print('Epoch {} test_loss is {}, test_accuracy is {}'.format(epoch,
                                                              test_loss.result(),
                                                              test_accuracy.result()))
        # 每一个batch结束后要清空
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [51]:
train()

Epoch0 loss is 1.0086686611175537, accuracy is 0.6808833479881287
Epoch 0 test_loss is 0.5173542499542236, test_accuracy is 0.8363000154495239
Epoch1 loss is 0.45733925700187683, accuracy is 0.8507833480834961
Epoch 1 test_loss is 0.371331125497818, test_accuracy is 0.8815000057220459
Epoch2 loss is 0.37708261609077454, accuracy is 0.8772000074386597
Epoch 2 test_loss is 0.3499488830566406, test_accuracy is 0.883899986743927
Epoch3 loss is 0.3356771767139435, accuracy is 0.8908166885375977
Epoch 3 test_loss is 0.2926556169986725, test_accuracy is 0.9075999855995178
Epoch4 loss is 0.3092726469039917, accuracy is 0.8995000123977661
Epoch 4 test_loss is 0.2968723773956299, test_accuracy is 0.902899980545044
Epoch5 loss is 0.2871173620223999, accuracy is 0.9061166644096375
Epoch 5 test_loss is 0.2576102018356323, test_accuracy is 0.9160000085830688
Epoch6 loss is 0.2718813717365265, accuracy is 0.9122333526611328
Epoch 6 test_loss is 0.23965752124786377, test_accuracy is 0.9221000075340271

### tf.keras.metrics 汇总计算模块

In [52]:
m = tf.keras.metrics.Mean('acc')

In [53]:
m(10)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [54]:
m(20)

<tf.Tensor: shape=(), dtype=float32, numpy=15.0>

In [55]:
m.result().numpy()

15.0

In [56]:
m([30, 40])

<tf.Tensor: shape=(), dtype=float32, numpy=25.0>

In [57]:
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=25.0>

In [58]:
m.reset_states()

In [59]:
m(1)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [60]:
m(2)

<tf.Tensor: shape=(), dtype=float32, numpy=1.5>

In [61]:
m.result().numpy()

1.5

In [62]:
# 汇总正确率
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')

In [63]:
a(labels, model(features))

<tf.Tensor: shape=(), dtype=float32, numpy=0.84375>

In [64]:
1/32

0.03125

In [65]:
#train()